In [1]:
# -*- coding: utf-8 -*-
import os

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#加载常量和前向传播函数
import inference


In [2]:
#配置神经网络的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 5000
MOVING_AVERAGE_DECAY = 0.99

#模型保存的路径和文件名
MODEL_SAVE_PATH = "./model/"
MODEL_NAME = "model"


In [3]:

def train(mnist):
    #定义输入输出placeholder
    x = tf.placeholder(tf.float32, [None, inference.INPUT_NODE], name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None, inference.OUTPUT_NODE], name = 'y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    #直接使用inference.py中的前向传播过程
    y = inference.inference(x, regularizer)
    global_step = tf.Variable(0, trainable = False)
    
    #损失函数、学习率、滑动平均以及训练过程
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y, labels = tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name = 'train')
        
    #初始化TensorFlow持久化类
    saver = tf.train.Saver()
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    
    with tf.Session(config=config) as sess:
        tf.global_variables_initializer().run()
        
        #在训练过程中不再测试模型在验证数据上的表现
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _,loss_value, step = sess.run([train_op, loss, global_step],feed_dict = {x:xs, y_:ys})
            #每1000轮保存一次模型
            if i%1000 == 0:
                #
                print("After %d training step(s), loss on training batch is %g."  %(step, loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH, MODEL_NAME),global_step=global_step)



In [4]:
def main(argv= None):
    mnist = input_data.read_data_sets("../MNIST_data", one_hot = True)
    train(mnist)
    
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 1 training step(s), loss on training batch is 2.90874.
After 1001 training step(s), loss on training batch is 0.238621.
After 2001 training step(s), loss on training batch is 0.195496.
After 3001 training step(s), loss on training batch is 0.137739.
After 4001 training step(s), loss

SystemExit: 

c:\users\lot\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
exit
